# Analyzing eCommerce Store Sales

# Executive Summary

## Introduction

This report provides an executive summary of the key findings from the analysis of sales data for an eCommerce store. The analysis focuses on various aspects such as visit patterns, channel grouping effectiveness, browser usage, operating system preferences, device categories, and geographical insights. The findings are derived from a series of plots that visualize the underlying data.

## Key Findings

**Visit Patterns**: The analysis of visit patterns indicates a significant variation in the number of visits across different hours of the day. Peak visit hours are observed during the evening, suggesting that most customers prefer shopping outside regular business hours.

**Channel Grouping Effectiveness**: The channel grouping analysis reveals that certain marketing channels are more effective in driving transactions than others. Organic search and direct channels lead in terms of transaction volume, highlighting the importance of SEO and brand recognition.

**Browser Usage**: Browser usage patterns suggest a diverse preference among users, with Chrome being the most popular browser. This indicates the necessity for the eCommerce store to ensure compatibility and optimization across various browsers to enhance user experience.

**Operating System Preferences**: The operating system analysis shows a clear preference for Windows and Android among users, suggesting that these platforms should be prioritized in terms of website optimization and app development to cater to the majority of the user base.

**Device Categories**: Device category analysis indicates a significant portion of traffic comes from mobile devices, emphasizing the importance of having a mobile-friendly website or app to accommodate the shopping preferences of modern consumers.

**Geographical Insights**: Geographical analysis highlights the global reach of the eCommerce store, with significant transactions originating from various continents and countries. North America and Europe emerge as key markets, suggesting potential areas for targeted marketing and expansion efforts.

## Conclusion

The analysis of the eCommerce store's sales data provides valuable insights into customer behavior, preferences, and the effectiveness of different marketing channels. By understanding these patterns, the store can optimize its marketing strategies, website, and app to better cater to its target audience, ultimately driving higher sales and customer satisfaction.

The sample dataset contains obfuscated GA360 data for August 1, 2017 from the [Google Merchandise Store](https://shop.googlemerchandisestore.com/), a real ecommerce store selling Google branded merchandise ([Source](https://support.google.com/analytics/answer/7586738)).

This data contains session data with traffic source, location and transcation info. Other data has been anonymized. The data is available for in a publicly available Google Sheet. We will be using a combination of SQL and Python to analyze this dataset, getting more insights into what is driving sales on this particular day.

In [119]:
-- This SQL query is designed to retrieve all rows from the ga_sessions table
SELECT *
FROM ga_sessions

,visitId,visitStartTime,visitNumber,channelGrouping,browser,operatingSystem,deviceCategory,continent,subContinent,country,transactions
0,1501591568,1501591568,1,Organic Search,Chrome,Windows,desktop,Europe,Southern Europe,Greece,NaN
1,1501589647,1501589647,2,Referral,Chrome,Windows,desktop,Asia,Southern Asia,India,NaN
2,1501616621,1501616621,1,Referral,Chrome,Windows,desktop,Europe,Northern Europe,United Kingdom,NaN
3,1501601200,1501601200,1,Referral,Firefox,Windows,desktop,Americas,Northern America,United States,NaN
4,1501615525,1501615525,1,Referral,Chrome,Windows,desktop,Americas,Northern America,United States,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2551,1501638264,1501638264,1,Referral,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2552,1501605330,1501605330,1,Organic Search,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2553,1501615026,1501615026,3,Organic Search,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2554,1501627131,1501627131,16,Organic Search,Chrome,Windows,desktop,Americas,Northern America,United States,1.0


In [120]:
#Display dataframe
df

,visitId,visitStartTime,visitNumber,channelGrouping,browser,operatingSystem,deviceCategory,continent,subContinent,country,transactions
0,1501591568,1501591568,1,Organic Search,Chrome,Windows,desktop,Europe,Southern Europe,Greece,NaN
1,1501589647,1501589647,2,Referral,Chrome,Windows,desktop,Asia,Southern Asia,India,NaN
2,1501616621,1501616621,1,Referral,Chrome,Windows,desktop,Europe,Northern Europe,United Kingdom,NaN
3,1501601200,1501601200,1,Referral,Firefox,Windows,desktop,Americas,Northern America,United States,NaN
4,1501615525,1501615525,1,Referral,Chrome,Windows,desktop,Americas,Northern America,United States,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2551,1501638264,1501638264,1,Referral,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2552,1501605330,1501605330,1,Organic Search,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2553,1501615026,1501615026,3,Organic Search,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2554,1501627131,1501627131,16,Organic Search,Chrome,Windows,desktop,Americas,Northern America,United States,1.0


In [121]:
# Import packages
import pandas as pd
import plotly.express as px

## Task 3: Clean the data

Let's inspect the types of the columns, and make adjustments where needed.

In [122]:
# Display the data types of each column in the dataframe 'df'
df.dtypes

visitId              int64
visitStartTime       int64
visitNumber          int64
channelGrouping     object
browser             object
operatingSystem     object
deviceCategory      object
continent           object
subContinent        object
country             object
transactions       float64
dtype: object

In [123]:
# Create a copy of the original dataframe to preserve the original data
df_clean = df.copy()

# Convert the 'visitStartTime' column from UNIX timestamp to datetime format
df_clean['visitStartTime'] = pd.to_datetime(df_clean['visitStartTime'], unit="s")

# Display the cleaned dataframe
df_clean

,visitId,visitStartTime,visitNumber,channelGrouping,browser,operatingSystem,deviceCategory,continent,subContinent,country,transactions
0,1501591568,2017-08-01 12:46:08,1,Organic Search,Chrome,Windows,desktop,Europe,Southern Europe,Greece,NaN
1,1501589647,2017-08-01 12:14:07,2,Referral,Chrome,Windows,desktop,Asia,Southern Asia,India,NaN
2,1501616621,2017-08-01 19:43:41,1,Referral,Chrome,Windows,desktop,Europe,Northern Europe,United Kingdom,NaN
3,1501601200,2017-08-01 15:26:40,1,Referral,Firefox,Windows,desktop,Americas,Northern America,United States,NaN
4,1501615525,2017-08-01 19:25:25,1,Referral,Chrome,Windows,desktop,Americas,Northern America,United States,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2551,1501638264,2017-08-02 01:44:24,1,Referral,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2552,1501605330,2017-08-01 16:35:30,1,Organic Search,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2553,1501615026,2017-08-01 19:17:06,3,Organic Search,Chrome,Macintosh,desktop,Americas,Northern America,United States,NaN
2554,1501627131,2017-08-01 22:38:51,16,Organic Search,Chrome,Windows,desktop,Americas,Northern America,United States,1.0


In [124]:
# Display the data types of each column in the dataframe 'df_clean'
df_clean.dtypes

visitId                     int64
visitStartTime     datetime64[ns]
visitNumber                 int64
channelGrouping            object
browser                    object
operatingSystem            object
deviceCategory             object
continent                  object
subContinent               object
country                    object
transactions              float64
dtype: object

## Task 4: Explore the data

Use the predefined plotting function to explore different grouped session counts as either a bar chart or a pie chart.

In [125]:
def plot_sessions_per_group(df, group, viz_type='bar'):
    # Group the dataframe by the specified column and count the number of sessions in each group
    sessions_per_group = df.groupby(group).size().reset_index(name='sessions').sort_values(by='sessions')
    
    # Check if the visualization type is 'bar'
    if viz_type == 'bar':
        # Create and return a bar chart showing the number of sessions per group
        return px.bar(sessions_per_group,
                      x=group,
                      y='sessions',
                      title=f'Number of sessions per {group}',
                      text='sessions')
    # Check if the visualization type is 'pie'
    elif viz_type == 'pie':
        # Create and return a pie chart showing the distribution of sessions per group
        return px.pie(sessions_per_group,
                      names=group,
                      values='sessions',
                      title=f'Distribution of sessions per {group}')
    else:
        # Raise an error if the visualization type is neither 'bar' nor 'pie'
        raise ValueError("viz_type can only be 'bar' or 'pie'")

In [126]:
# Plot the number of sessions per continent using the predefined plotting function
plot_sessions_per_group(df_clean, 'continent')

![newplot (4)](newplot%20(4).png)


In [127]:
# Plot the number of sessions per channel grouping using the predefined plotting function
plot_sessions_per_group(df_clean, 'channelGrouping')

![newplot (3)](newplot%20(3).png)


In [128]:
# Plot the distribution of sessions per device category using a pie chart
plot_sessions_per_group(df_clean, 'deviceCategory', 'pie')

![newplot (2)](newplot%20(2).png)


In [129]:
# Initialize timezone for further conversion
df_clean['visitStartTime'] = df_clean['visitStartTime'].dt.tz_localize('utc')

# Convert the 'visitStartTime' column to the 'America/Los_Angeles' timezone
df_clean['visitStartTime'] = df_clean['visitStartTime'].dt.tz_convert('America/Los_Angeles')

# Create a new column 'hour' by extracting the hour from 'visitStartTime' and converting it to a period of type 'H' (hourly)
df_clean['hour'] = df_clean['visitStartTime'].dt.to_period('H')

# Group the data by 'hour' and 'channelGrouping', count the number of visits in each group, and reset the index to make 'visits' a column
visits_per_month = df_clean.groupby(['hour', 'channelGrouping']).size().reset_index(name='visits')

# Convert the 'hour' column to string to facilitate plotting
visits_per_month['hour'] = visits_per_month['hour'].astype(str)

# Plot the number of visits per hour throughout the day for each channel grouping using a line chart with markers
px.line(visits_per_month, x='hour', y='visits', color='channelGrouping', title='Visits per hour throughout the day by Channel', markers=True)

![newplot (1)](newplot%20(1).png)


In [130]:
# Group the cleaned dataframe by 'channelGrouping' and aggregate the sum of transactions and the count of visitId
cgs = (
    df_clean
    .groupby('channelGrouping')
    .agg({'transactions': 'sum', 'visitId': 'size'})
    .reset_index()
)

# Calculate the transactions per session by dividing the sum of transactions by the count of visitId
cgs['trans_per_session'] = cgs['transactions'] / cgs['visitId']

# Sort the dataframe by 'trans_per_session' in ascending order
cgs = cgs.sort_values(by='trans_per_session')

# Plot the transactions per session for each channel grouping on a bar chart
px.bar(cgs,
      x='channelGrouping',
      y='trans_per_session',
      title='Transactions per sessions by Channel'
      )

![trans_per_session](data/trans_per_session.png)
